In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from helpers import read_from_csv, plot_array
from models import correlation_model

In [ ]:
gene_df = read_from_csv('gene-data.csv')

In [ ]:
gene_df.head()

In [ ]:
ground_truth = np.array([[0,1,0,0,0],
                         [0,0,1,0,0],
                         [0,0,0,0,1],
                         [0,0,0,0,0],
                         [1,1,0,1,0]])

In [ ]:
corr_mat = pd.DataFrame(correlation_model(gene_df, 0.5))

In [ ]:
plot_array(corr_mat)